In [2]:
from urllib import response
from attr import Attribute
import boto3 as bt
import json
import cx_Oracle as cx
import tornado.web as tw
import tornado.ioloop
from tornado.escape import json_decode
import json
import requests
import cx_Oracle as cx
import pandas as pd

In [4]:
s3 = bt.resource("s3")

# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)


s3-bucket-notify


In [ ]:

def create_queue():
    sqs_client = bt.client("sqs",region_name="ap-south-1")
    response = sqs_client.create_queue(
        QueueName = "bk_queue",
        Attributes={
            "DelaySeconds":"0",
            "VisibilityTimeout":"60",
            "FifoQueue":True
        }
    )
    print(response)
    


In [ ]:
create_queue()

In [ ]:
def get_queue_url(name):
    sqs_client = bt.client("sqs",region_name="ap-south-1")
    response = sqs_client.get_queue_url(
        QueueName = name,
    )
    return response["QueueUrl"]


print(get_queue_url("poll.fifo"))

In [ ]:
def send_message():
    sqs_client = bt.client("sqs",region_name="ap-south-1")
    message = {"name" : "Mudit" , "age" : 22, "location" : "Noida"}
    response = sqs_client.send_message(
        QueueUrl = "https://sqs.ap-south-1.amazonaws.com/884379823401/testQueue",
        MessageBody = json.dumps(message)
    )
    # print(response)
    
send_message()

In [3]:
def recieve_message(url):
    sqs_client = bt.client("sqs",region_name="ap-south-1")
    response = sqs_client.receive_message(
        QueueUrl = url,
        MaxNumberOfMessages=3,
        WaitTimeSeconds=0
    )
    print(response)
    # print(f"Number of messages recieved :{len(response.get('Messages',[]))}")
    # lst = []
    # for message in response.get("Messages",[]):
    #     message_body = message["Body"]
    #     print(f"Message body:{json.loads(message_body)}")
    #     print(f"Receipt Handle:{message['ReceiptHandle']}")
    #     # print(message['ReceiptHandle'])
    # # return response.get('ReceiptHandle')
# val = recieve_message("https://ap-south-1.queue.amazonaws.com/884379823401/bk_queue")
# val2 = val["Messages",[]]
# msg  =val.get("Messages",[])
# print(msg)
# msg_body = msg[0]
# print(msg_body["Body"])
recieve_message("https://sqs.ap-south-1.amazonaws.com/884379823401/testQueue")

{'Messages': [{'MessageId': 'c06d3ca4-b248-495f-92df-42ba7331bd58', 'ReceiptHandle': 'AQEBcrzH0j2EYLkHsm7FRfVgO/B0kdbhtoeZtAVCfWb1uSVthgkNeE+OGW76Bughs+2shFKqTrnN8k6rOwRVHkZIpZ6afj1mnoktbimDqwbLDYdpderG8TwuGDto5qhdNilwvRJPzxP9+67gp8Ng+wYSItNgBNRut9udlCBnD/PKtPDhXkoNz4qQM10X5yFpKjmTbKxo6DnXTUB/pg7lMJYoVejS0BKrlYBCb60B9k7Na507eAQU6jE3mtO5vqQag2UTgtXYu5GcXdGMyeQPVMcDUiY3xNDfEr3qQOHAZZN0CnCG+LAQcFKrw0Qmp/dDCVVQKnI/y6asgARJlfBhyGNQzxi1NIXwzJ/1Y0Z4KIe6Ty0CH62W0mfIQYVL4uj6A6uGtgvL0ZyCi4IuF67mvsbozA==', 'MD5OfBody': '3dd9fb1cc6b7efa962d044bca1c6fd3f', 'Body': '{"Records":[{"eventVersion":"2.1","eventSource":"aws:s3","awsRegion":"ap-south-1","eventTime":"2022-08-22T09:46:58.278Z","eventName":"ObjectCreated:Put","userIdentity":{"principalId":"A28SD5F053SAJD"},"requestParameters":{"sourceIPAddress":"182.76.1.86"},"responseElements":{"x-amz-request-id":"WGRDR71H3KTHEST5","x-amz-id-2":"9JKXvYjq7Gu8d0nkcqctwC+aA3ect2Ou22rt/NpH+aQgmKzNc66aspgOc7F8KygdReJsaWQYmeQ9blzOX87yunnqMzGNCGsh"},"s3":{"s3Schema

In [ ]:
def delete_message(reciept_handle,url):
    sqs_client = bt.client("sqs",region_name="ap-south-1")
    response = sqs_client.delete_message(
        QueueUrl = url,
        ReceiptHandle =  reciept_handle,
    )
    # print(response)

if __name__=="__main__":
    # "https://ap-south-1.queue.amazonaws.com/884379823401/bk_queue"
    val = recieve_message()
    msg  =val.get("Messages",[])
    receiptHandle = msg[0]
    print(receiptHandle["ReceiptHandle"])
    delete_message(receiptHandle["ReceiptHandle"],"https://ap-south-1.queue.amazonaws.com/884379823401/bk_queue")

In [ ]:
def process_msg(QueueName)->list:
    QueueUrl = get_queue_url(QueueName)
    response =  recieve_message(QueueUrl)
    lst = []
    for message in response.get("Messages",[]):
        message_body = message["Body"]
        lst.append(message_body)
    return lst
    

In [ ]:
msg_box = []
msg_box = process_msg("bk_queue")
v = requests.post("http://127.0.0.1:8001/",data=msg_box[0])

In [ ]:
# # still to implement DLQ(dead letter queue)




# class reqHandler(tornado.web.RequestHandler): 
#     def post(self):
#         print(self.request.body)
#         byte_value2 = json.loads(self.request.body) 
#         s = json.dumps(byte_value2,indent=4)
#         cdns = cx.makedsn('10.100.22.99','1521',service_name='PVSDEVDB')
#         cnxn = cx.connect(user=r'PVS_DB_55JULY', password='rxlogix', dsn=cdns)
#         c = cnxn.cursor()
#         c.execute(f"insert into tor_test(NAME,AGE,LOCATION) values('{byte_value2['name']}','{byte_value2['age']}','{byte_value2['location']}')")
#         cnxn.commit()
#         cnxn.close()  
#     def set_default_headers(self):
#         self.set_header('Access-Control-Allow-Methods', 'POST, GET, OPTIONS')
#         self.set_header('Access-Control-Allow-Headers',
#                         'Content-Type, Access-Control-Allow-Origin, Access-Control-Allow-Headers, X-Requested-By, Access-Control-Allow-Methods')